In [ ]:
#Unzip the files

import zipfile

with zipfile.ZipFile('/Users/paramanandbhat/Downloads/train_nLPp5K8.zip', 'r') as zip_ref:
    zip_ref.extractall(".")


import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('/Users/paramanandbhat/Downloads/test_fkwGUNG.csv')
base_path = '/Users/paramanandbhat/Downloads/train_nLPp5K8/images'
from tensorflow.keras.preprocessing.image import ImageDataGenerator
!pip install scikit-learn

from sklearn.model_selection import train_test_split

# Create a train-validation split
train_df['image_path'] = train_df['image_names'].apply(lambda x: f'{base_path}/{x}.jpg')
train, val = train_test_split(train_df, test_size=0.2)


train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)


# Convert the 'class' column to string type
train_df['class'] = train_df['class'].astype(str)

# Now, create the train_generator with the modified DataFrame
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',  # Ensure this column contains the full path to the images
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

# Update the image paths in the DataFrame
train_df['image_path'] = '/Users/paramanandbhat/Downloads/train_nLPp5K8/images/' + train_df['image_names'].astype(str)
# Check the first few image paths
print(train_df['image_path'].head())

# Verify the existence of the first image file
import os
first_image_path = train_df['image_path'].iloc[0]
print(f"Does the first image file exist? {os.path.exists(first_image_path)}")
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Create a train and val split
# Create a train-validation split
train, val = train_test_split(train_df, test_size=0.2)
val_datagen = ImageDataGenerator(rescale=1./255)

# Assuming train_datagen has already been defined with the desired configurations
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Example, adjust according to your needs

# Create the train_generator using the flow_from_dataframe method
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train,
    directory=None,
    x_col='image_path',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')


validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val,
    directory=None,
    x_col='image_path',
    y_col='class',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

#building and training a simple convolutional neural network (CNN) model for your gender classification task using TensorFlow and Keras
#Step 1: Import Necessary Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

#Step 2: Define Your Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),  # Dropout layer to reduce overfitting
    Dense(1, activation='sigmoid')  # Sigmoid activation for binary classification
])

from tensorflow.keras.optimizers import legacy

# Use the legacy Adam optimizer for potentially better performance on M1/M2 Macs
model.compile(optimizer=legacy.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    verbose=1
)
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
import pandas as pd
import os

# Load the test.csv file
test_df = pd.read_csv('/Users/paramanandbhat/Downloads/test_fkwGUNG.csv')

#  test images are in the same folder structure as  training images
test_images_directory = '/Users/paramanandbhat/Downloads/train_nLPp5K8/images/'

# Construct the full image paths
test_df['image_path'] = test_images_directory + test_df['image_names'].astype(str)

from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

# Preprocess function
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.mobilenet_v2.preprocess_input(img_array_expanded_dims)

# Preprocess all test images
preprocessed_images = np.vstack([preprocess_image(path) for path in test_df['image_path']])

predictions = model.predict(preprocessed_images)
# Assuming your model outputs probabilities for being in one class, e.g., female
# Convert probabilities to binary class labels based on a threshold
predicted_classes = (predictions > 0.5).astype(int).reshape(-1)

# Add the predicted classes to test_df
test_df['class'] = predicted_classes

# If required, adjust column names and order to match the submission format
submission_df = test_df[['image_names', 'class']]

# Save the DataFrame to a new CSV file
submission_df.to_csv('/Users/paramanandbhat/Downloads/train_nLPp5K8/submissionfinal.csv', index=False)

